In [77]:
from iiif_prezi3 import Manifest, AnnotationPage, Annotation, ResourceItem, config
from moviepy.editor import VideoFileClip
import iiif_prezi3

In [78]:
def get_video_duration(filename):
    with VideoFileClip(filename) as video:
        return video.duration

In [79]:
def create(item_id, metadata):

    attribution = "国立国会図書館 National Diet Library, JAPAN"
    rights = "https://www.ndl.go.jp/jp/use/reproduction/index.html"
    homepage = "https://rekion.dl.ndl.go.jp/"

    dirname = "demo"

    pages_prefix = f"https://nakamura196.github.io/ramp_data/{dirname}"

    label = metadata.get("label", "Unknown")

    prefix = f"{pages_prefix}/{item_id}"
    output_path = f"../../../docs/{dirname}/{item_id}/manifest.json"
    mp4_path = f"../../../docs/{dirname}/{item_id}/{item_id}.mp4"
    mp4_url = f"{pages_prefix}/{item_id}/{item_id}.mp4"
    vtt_url = f"{pages_prefix}/{item_id}/{item_id}.vtt"

    config.configs['helpers.auto_fields.AutoLang'].auto_lang = "ja"

    duration=get_video_duration(mp4_path)

    metadata_fixed = []

    for key, values in metadata.items():
        if key == "label":
            continue
        for value in values:
            metadata_fixed.append(iiif_prezi3.KeyValueString(label=key, value=value))


    manifest = Manifest(
        id=f"{prefix}/manifest.json", 
        label=label,
        metadata=metadata_fixed,
        rights=rights,
        requiredStatement=iiif_prezi3.KeyValueString(label="Attribution", value=attribution),
        provider=iiif_prezi3.ProviderItem(
                    id=homepage,
                    label=attribution,
                ),
                homepage=iiif_prezi3.HomepageItem(
                    id=f"https://rekion.dl.ndl.go.jp/pid/{item_id}",
                    type="Text",
                    label=f"{label} - 国立国会図書館デジタルコレクション",
                    format="text/html",
                    language="ja"
                ),
        )
    canvas = manifest.make_canvas(id=f"{prefix}/canvas", duration=duration)
    anno_body = ResourceItem(id=mp4_url,
                            type="Sound",
                            format="audio/mp4",
                            duration=duration)
    anno_page = AnnotationPage(id=f"{prefix}/canvas/page")
    anno = Annotation(id=f"{prefix}/canvas/page/annotation",
                    motivation="painting",
                    body=anno_body,
                    target=canvas.id)
    anno_page.add_item(anno)
    canvas.add_item(anno_page)

    # VTT URLを追加
    vtt_body = ResourceItem(id=vtt_url, type="Text", format="text/vtt")
    vtt_anno = Annotation(
        id=f"{prefix}/canvas/annotation/webvtt",
        motivation="supplementing",
        body=vtt_body,
        target=canvas.id,
        label = "WebVTT Transcript (machine-generated)"
        )

    vtt_anno_page = AnnotationPage(id=f"{prefix}/canvas/page/2")
    vtt_anno_page.add_item(vtt_anno)

    canvas.annotations = [vtt_anno_page]

    with open(output_path, "w") as f:
        f.write(manifest.json(indent=2))


In [80]:
metadata = {
    "3574643": {
      "label": "講演：道徳、経済合一論（一）Union of Morality of Economy",
      "作詞・作曲・編曲・実演家": [
        "Shibusawa（渋沢　栄一）[作詞]",
        "Shibusawa（渋沢　栄一）"
      ],
      "製作者（レーベル）": [
        "ニッポノホン"
      ],
      "注記": [
        "商品番号 : 15289",
        "デジタル変換後ノイズ除去 : ノイズ除去なし",
        "講演"
      ],
      "出版年月日(W3CDTF)": [
        "0000"
      ],
      "歴史的音源ジャンル": [
        "講義、講演、演説"
      ]
  },
  "3571280": {
      "label": "日本のアクセントと言葉調子（下）",
      "作詞・作曲・編曲・実演家": [
        "神保　格"
      ],
      "製作者（レーベル）": [
        "コロムビア（戦前）"
      ],
      "注記": [
        "商品番号 : 33000",
        "デジタル変換後ノイズ除去 : 無",
        "日本語学習"
      ],
      "出版年月日(W3CDTF)": [
        "1929-09"
      ],
      "歴史的音源ジャンル": [
        "教育・児童",
        "語学"
      ]
  }
}

item_ids = ["3571280", "3574643"]

for item_id in item_ids:
    create(item_id, metadata[item_id])